In [1]:
print()

In [2]:
# LOAD PACKAGES
# -------------
# import Pkg; Pkg.add("CSV"); Pkg.add("GLPK"); Pkg.add("JuMP")
using JuMP  # Need to say it whenever we use JuMP
using GLPK  # Loading the GLPK module for using its solver
using CSV

In [10]:
# MODEL CONSTRUCTION
# ------------------
capstone = Model(with_optimizer(GLPK.Optimizer, tm_lim=10*1000)) 
# We also set the time limit, in milliseconds

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [15]:
# INPUT DATA
# ----------

# Data on student preferences
unit_costs = CSV.read("../preprocessing/unit_costs.csv",header=false)
print(size(unit_costs))

# Data on Pillars
capacities = CSV.read("../preprocessing/capacities.csv",header=false)
print(size(capacities))

# Data on Pillars
supplies = CSV.read("../preprocessing/supplies.csv",header=false)
print(size(supplies))

length = size(unit_costs)[1]-1
iterlen = 1:length

(6, 6)(6, 6)(6, 1)

1:5

In [16]:
# VARIABLES
# ---------

# the flow
@variable(capstone, flow[i=iterlen,j=iterlen] >= 0, Int)  

# binary variable taking value 1 if there is a flow
@variable(capstone, flowing[i=iterlen,j=iterlen], Bin)

print()

In [17]:
# CONSTRAINTS
# -----------

# flow can be nonzero only if flowing is 1
@constraint(capstone, smaller_1[i=iterlen, j=iterlen], 
    flow[i,j] <= flowing[i,j]*9999999)

# flow can be nonzero only if capacity is nonzero
@constraint(capstone, smaller_2[i=iterlen, j=iterlen],
    flowing[i,j] <= capacities[i,j])

# # Every node can only have one output
@constraint(capstone, output[i=iterlen],
    sum(flowing[i,j] for j = iterlen) <= 1)

# # Every node needs to exhaust its supply
@constraint(capstone, supplied[i=iterlen],
    sum(flow[i,j] for j = iterlen) - sum(flow[j,i] for j = iterlen) >= supplies[i,1])

print()

In [22]:
# OBJECTIVE
# ---------

# Maximize the total utility
@objective(capstone, Min, 
    sum(unit_costs[i,j]*flow[i,j] for i = iterlen, j = iterlen))

print(capstone) # Shows the model constructed in a human-readable form
print()

Min 0.13071583043221874 flow[1,1] + 0.645553129731741 flow[1,2] + 0.14590703656024112 flow[1,3] + 0.40875024711491015 flow[1,4] + 1.0756329242588578 flow[1,5] + 0.2598680125917515 flow[2,1] + 0.5466734446190455 flow[2,2] + 0.6123989363728397 flow[2,3] + 1.2162056648110726 flow[2,4] + 1.3174071575250084 flow[2,5] + 0.08435989323537346 flow[3,1] + 0.47080458722650975 flow[3,2] + 1.1423938319129114 flow[3,3] + 0.17293122874903874 flow[3,4] + 0.5845396978256493 flow[3,5] + 1.0973674708141525 flow[4,1] + 0.2864354408696484 flow[4,2] + 0.4625145506838141 flow[4,3] + 1.6149406129858435 flow[4,4] + 0.15652769681523246 flow[4,5] + 0.4323884807130002 flow[5,1] + 0.49768267603182376 flow[5,2] + 0.5589519324648493 flow[5,3] + 0.48972247119015117 flow[5,4] + 2.579328067174117 flow[5,5]
Subject to
 flow[1,2] + flow[1,3] + flow[1,4] + flow[1,5] - flow[2,1] - flow[3,1] - flow[4,1] - flow[5,1] ≥ 1.0
 flow[2,1] + flow[2,3] + flow[2,4] + flow[2,5] - flow[1,2] - flow[3,2] - flow[4,2] - flow[5,2] ≥ 2.0
 fl

In [19]:
# SOLVE IT
# --------

status = optimize!(capstone)

In [20]:
# DISPLAY THE RESULTS
# -------------------

println("Objective value: ", JuMP.objective_value(capstone)) 

MathOptInterface.ResultIndexBoundsError{MathOptInterface.ObjectiveValue}: Result index of attribute MathOptInterface.ObjectiveValue(1) out of bounds. There are currently 0 solution(s) in the model.